# Lab | Summarization evaluation using LangSmith
Let's revisit your capstone project 2? Well, sort of. Pick diffierent sets of data and re-run this notebook. Maybe parts of the dataset you used in your last project week. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications using LangSmith.

What did you learn? - Let's discuss that in class

## LangSmith - LangChain evaluation

In [2]:
%pip install dotenv

In [18]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [67]:

from google.colab import userdata
userdata.get('LANGCHAIN_API_KEY')
HUGGINGFACEHUB_API_TOKEN = userdata.get('HF_TOKEN')

In [19]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="langsmith_max-test"

In [5]:
#Importing Client from Langsmith
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


### Create Dataset


In [7]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [8]:
from datasets import load_dataset
cnn_dataset = load_dataset(
    "ccdv/cnn_dailymail", version
    ="3.0.0",
    trust_remote_code=True
)

README.md:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

cnn_dailymail.py:   0%|          | 0.00/9.27k [00:00<?, ?B/s]

cnn_stories.tgz:   0%|          | 0.00/159M [00:00<?, ?B/s]

dailymail_stories.tgz:   0%|          | 0.00/376M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
def add_prefix(example):
    return {
        **example,
        "article": f"Summarize this news:\n{example['article']}"
    }

#cnn_dataset = cnn_dataset.map(add_prefix)

In [10]:
cnn_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [11]:
cnn_dataset['train'][0]

{'article': 'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but bec

In [12]:
#Get just a few news to test
MAX_NEWS=10
sample_cnn = cnn_dataset["test"].select(range(MAX_NEWS)).map(add_prefix)

sample_cnn

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 10
})

The dataset contains three columns: article, highlights, and id. To use LangSmith, we need to create a dataset in LangSmith format.

LangSmith expects a prompt and a result. To achieve this, we will transform the article into a prompt by adding the prefix: "Summarize this news." As a result, we will use the content of highlights, which represents the summaries created by humans.

In [13]:
print(sample_cnn[0])

{'article': 'Summarize this news:\n(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV\'s "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he\'d been a busy actor for decades in theater and in Hollywood, Best didn\'t become famous until 1979, when "The Dukes of Hazzard\'s" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best\'s Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle

Now We have the Dataset with the prompt and the Reference Summary, it is time to create a Dataset in LangSmith with this information.
### Create the Dataset in Langsmith

The dataset in LangSmith is composed of an input, which is the prompt passed to the model for evaluation, and an output, which should contain what we expect the model to return.

In [14]:
import datetime

In [15]:
import uuid
input_key=['article']
output_key=['highlights']

NAME_DATASET=f"Summarize_dataset_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

In [27]:
#This creates the dataset in LangSmith with the content in sample_cnn - If you run this more than once you will get POST errors
client = Client(api_key=userdata.get('LANGCHAIN_API_KEY'))
dataset = client.upload_dataframe(
    df=sample_cnn,
    input_keys=input_key,
    output_keys=output_key,
    name=NAME_DATASET,
    description="Test Embedding distance between model summarizations",
    data_type="kv"
)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In this image, we can see an example from the dataset once it's been registered in LangSmith.

In the Input column, there is the prompt to be sent, while in the Output column, the expected output is stored.

When performing the comparison, the model will be given the prompt, and the Cosine distance between its response and the one stored in the sample dataset will be calculated.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Dataset.jpg?raw=true">

### Recovering Models From Hugging Face
Let's retrieve both models from HuggingFace. A base T5 model and a model that has been fine-tuned using the training portion of this same dataset to generate summaries.

In [29]:
%pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.7 MB/s eta 0:00:00


In [30]:
from langchain import HuggingFaceHub

In [33]:
summarizer_base = HuggingFaceHub(
    repo_id="t5-base",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

In [34]:
summarizer_finetuned = HuggingFaceHub(
    repo_id="flax-community/t5-base-cnn-dm",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

## Defining Evaluator
The first step is to define an evaluator, where we specify the variables we want to evaluate. In our case, I have chosen to measure only the "embedding_distance."

I've left the "string_distance" as a comment in case you want to conduct a test with two evaluations instead of one.


In [37]:
from langchain.smith import run_on_dataset, RunEvalConfig
!pip install -q rapidfuzz==3.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 64.0 MB/s eta 0:00:00


In [38]:
#We are using just one of the multiple evaluator avaiable on LangSmith.

evaluation_config = RunEvalConfig(
    evaluators=[
        "embedding_distance",
        "string_distance"
    ],
)



### Running Evaluator
With the same configuration, we can launch two evaluations on the same dataset. One for each of the chosen models.

In [76]:
import openai
# Detect if the environment is Colab
try:
    import google.colab  # If in Colab
    is_colab = True
except ImportError:
    is_colab = False

# Set your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')


In [78]:
%pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.6 MB/s eta 0:00:00


In [79]:
project_name = f"T5-BASE {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

base_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_base,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-BASE 2025-04-25 18:52:48' at:
https://smith.langchain.com/o/8de1eb8c-9022-4abe-94b7-2506c500f049/datasets/a3e41f43-e718-4345-9523-885c1937b6f9/compare?selectedSessions=8fbb2525-ca5c-432d-b64b-b73c425a34c5

View all tests for Dataset Summarize_dataset_2025-04-25 18:19:18 at:
https://smith.langchain.com/o/8de1eb8c-9022-4abe-94b7-2506c500f049/datasets/a3e41f43-e718-4345-9523-885c1937b6f9
[>                                                 ] 0/10

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_

[---->                                             ] 1/10

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
Error Type: HfHubHTTPError, Message: (Request ID: Root=1-680bda03-409a7f254911ec4640acf12b;368711bf-4d21-428b-bcdb-9dc58e1cd38b)

403 Forbidden: This authentication method does not have sufficient permissions to call Inference Providers on behalf of user R5D.
Cannot access content at: https://router.huggingface.co/hf-inference/models/t5-base.
Make sure your token has the correct permissions.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarn

[-------------->                                   ] 3/10

...

[---------------------------------->               ] 7/10

Error Type: HfHubHTTPError, Message: (Request ID: Root=1-680bda07-2e3a777a294450313f4fd998;d91eaddb-e74a-44b8-a3b1-b240cacd5df7)

403 Forbidden: This authentication method does not have sufficient permissions to call Inference Providers on behalf of user R5D.
Cannot access content at: https://router.huggingface.co/hf-inference/models/t5-base.
Make sure your token has the correct permissions.
Error Type: HfHubHTTPError, Message: (Request ID: Root=1-680bda08-3b2f340c017c994c2d4a8668;2fe93cee-7dcb-49fc-8800-20e97e18b6e4)

403 Forbidden: This authentication method does not have sufficient permissions to call Inference Providers on behalf of user R5D.
Cannot access content at: https://router.huggingface.co/hf-inference/models/t5-base.
Make sure your token has the correct permissions.


[-------------------------------------------->     ] 9/10

Error Type: HfHubHTTPError, Message: (Request ID: Root=1-680bda08-1dd1ddd533882e85278e8d7e;a6f33c71-a92d-43b7-bc91-f8e13432de5b)

403 Forbidden: This authentication method does not have sufficient permissions to call Inference Providers on behalf of user R5D.
Cannot access content at: https://router.huggingface.co/hf-inference/models/t5-base.
Make sure your token has the correct permissions.


[------------------------------------------------->] 10/10

In [80]:
#Ignore the error shown below
project_name = f"T5-FineTuned {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_finetuned,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-FineTuned 2025-04-25 18:58:55' at:
https://smith.langchain.com/o/8de1eb8c-9022-4abe-94b7-2506c500f049/datasets/a3e41f43-e718-4345-9523-885c1937b6f9/compare?selectedSessions=cfc135e2-353b-4832-b49e-36ad42497fff

View all tests for Dataset Summarize_dataset_2025-04-25 18:19:18 at:
https://smith.langchain.com/o/8de1eb8c-9022-4abe-94b7-2506c500f049/datasets/a3e41f43-e718-4345-9523-885c1937b6f9
[>                                                 ] 0/10

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_

[---->                                             ] 1/10

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
Error Type: HfHubHTTPError, Message: (Request ID: Root=1-680bdb71-6b5a4953467c37b61498594b;18b6a553-13a9-45af-b62a-6733dfcd351f)

403 Forbidden: This authentication method does not have sufficient permissions to call Inference Providers on behalf of user R5D.
Cannot access content at: https://router.huggingface.co/hf-inference/models/flax-community/t5-base-cnn-dm.
Make sure your token has the correct permissions.


[--------->                                        ] 2/10

...

[------------------------>                         ] 5/10

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
Error Type: HfHubHTTPError, Message: (Request ID: Root=1-680bdb73-7ae506da4f17f96b65b6af07;a6b85d96-1ed5-4a1f-8520-847327eb50b3)

403 Forbidden: This authentication method does not have sufficient permissions to call Inference Providers on behalf of user R5D.
Cannot access content at: https://router.huggingface.co/hf-inference/models/flax-community/t5-base-cnn-dm.
Make sure your token has the correct permissions.
Error Type: HfHubHTTPError, Message: (Request ID: Root=1-680bdb73-0df86

[---------------------------------->               ] 7/10

Error Type: HfHubHTTPError, Message: (Request ID: Root=1-680bdb74-25a14a116b67cad609737951;fbc9ed5e-641c-4d51-aebb-535f08b191bc)

403 Forbidden: This authentication method does not have sufficient permissions to call Inference Providers on behalf of user R5D.
Cannot access content at: https://router.huggingface.co/hf-inference/models/flax-community/t5-base-cnn-dm.
Make sure your token has the correct permissions.
Error Type: HfHubHTTPError, Message: (Request ID: Root=1-680bdb74-731a3b9479f59e842dd1073a;7dcda538-a73b-4ad6-8097-c391d280f203)

403 Forbidden: This authentication method does not have sufficient permissions to call Inference Providers on behalf of user R5D.
Cannot access content at: https://router.huggingface.co/hf-inference/models/flax-community/t5-base-cnn-dm.
Make sure your token has the correct permissions.
Error Type: HfHubHTTPError, Message: (Request ID: Root=1-680bdb74-1a1d4d133b62ca4229ff77b9;e57e0aa9-8dd2-4a95-9d8d-512390165582)

403 Forbidden: This authentication m

[------------------------------------------------->] 10/10

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Tests.jpg?raw=true">

In the image below you can see the comparision between two tests.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareTestst.jpg?raw=true">

Well, since it has been so straightforward, why don't we try to make the comparison with an OpenAI model?

In [81]:
%pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.2 MB/s eta 0:00:00


In [83]:
from langchain_openai import OpenAI
open_aillm=OpenAI(temperature=0.0)

In [84]:
project_name = f"OpenAI {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=open_aillm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'OpenAI 2025-04-25 18:59:57' at:
https://smith.langchain.com/o/8de1eb8c-9022-4abe-94b7-2506c500f049/datasets/a3e41f43-e718-4345-9523-885c1937b6f9/compare?selectedSessions=b5d0209d-66f1-4a76-8b7d-762cc70ceb19

View all tests for Dataset Summarize_dataset_2025-04-25 18:19:18 at:
https://smith.langchain.com/o/8de1eb8c-9022-4abe-94b7-2506c500f049/datasets/a3e41f43-e718-4345-9523-885c1937b6f9
[------------------------------------------------->] 10/10

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareOpenAI_HF.jpg?raw=true">

The experiment with the OpenAI model has yielded the best results. But, be aware! As we can see, there is a cost involved since we are using an API, and it needs to be paid for.

Another crucial piece of information is that we can view performance data for the models. This data could also be useful for minimally evaluating our inference server.